# Tutoriel Python - MDSF 2018

Ce tutoriel a pour but de guider les personnes souhaitant utiliser Python pour participer au challenge.

Il comporte 5 étapes :

1. Import des données
2. Analyse descriptive
3. Préparation des données
4. Création d’un modèle
5. Calcul des prédictions et soumissions

# Import des données

Avant de rentrer dans le vif du sujet, installons les packages nécessaires pour ce tutoriel :

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
pd.set_option('display.max_columns', 500)

In [36]:
%%time
X_train = pd.read_csv("train.csv", index_col=0, error_bad_lines=False)
X_test = pd.read_csv("test.csv", index_col=0, error_bad_lines=False)

Wall time: 23.6 s


In [37]:
X_test.head(3)
X_train.head(3)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
item_id,,,,,,,,,,,,,,,,,
b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177


In [38]:
y_train = X_train["deal_probability"]

In [39]:
print("Dimension X_train:", X_train.shape)
print("Dimension X_test:", X_test.shape)

Dimension X_train: (1503424, 17)
Dimension X_test: (508438, 16)


# Analyse descriptive

## Structure des datasets

Le dataset train comporte les caractéristiques et délai de vente de **8880 objets** vendus sur le site Emmaus. C’est ce dataset que nous allons utiliser pour créer un modèle. Chaque objet est décrit par une observation de X variables. Ces variables sont décrites dans le fichier ```description.pdf``` présent dans la clef USB.

Le dataset test comporte les caractéristiques des **2960 objets** dont il faut prédire le délai de vente. A la différence du train, le délai de vente n’est bien sûr pas renseigné et une colonne ```id``` a ete rajoutée pour identifier les prédictions pendant l’étape de soumission.

In [40]:
X_train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
user_id,1503424,771769,45ba3f23bf25,1080,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region,1503424,28,Краснодарский край,141416,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city,1503424,1733,Краснодар,63638,NaN,NaN,NaN,NaN,NaN,NaN,NaN
parent_category_name,1503424,9,Личные вещи,697623,NaN,NaN,NaN,NaN,NaN,NaN,NaN
category_name,1503424,47,"Одежда, обувь, аксессуары",282753,NaN,NaN,NaN,NaN,NaN,NaN,NaN
param_1,1441848,371,Женская одежда,226289,NaN,NaN,NaN,NaN,NaN,NaN,NaN
param_2,848882,271,Обувь,150450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
param_3,640859,1219,Вторичка,50615,NaN,NaN,NaN,NaN,NaN,NaN,NaN
title,1503424,788377,Платье,15550,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,1387148,1317102,В хорошем состоянии,2249,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
X_test.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
user_id,508438,306069,abcdd1ca1a5c,496,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region,508438,28,Краснодарский край,45098,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city,508438,1644,Екатеринбург,22391,NaN,NaN,NaN,NaN,NaN,NaN,NaN
parent_category_name,508438,9,Личные вещи,216577,NaN,NaN,NaN,NaN,NaN,NaN,NaN
category_name,508438,47,"Одежда, обувь, аксессуары",84896,NaN,NaN,NaN,NaN,NaN,NaN,NaN
param_1,485528,361,Женская одежда,66642,NaN,NaN,NaN,NaN,NaN,NaN,NaN
param_2,275209,255,Обувь,48776,NaN,NaN,NaN,NaN,NaN,NaN,NaN
param_3,202107,973,Вторичка,18307,NaN,NaN,NaN,NaN,NaN,NaN,NaN
title,508438,298039,Платье,4673,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,508438,485209,В хорошем состоянии,918,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Le jeu de données est très équilibré, chacune des 3 classes a une fréquence proche d’1/3.

# Création d'un modèle

Il est maintenant temps de créer un modele. Dans ce tutoriel nous allons construire une [Forêt Aléatoire](https://fr.wikipedia.org/wiki/For%C3%AAt_d'arbres_d%C3%A9cisionnels)

Pour ce faire nous utilisons les variables ```["poids", "prix", "nb_images", "longueur_image", "largeur_image", "categorie"]```.

Pour éviter le [surapprentissage](https://fr.wikipedia.org/wiki/Surapprentissage) et estimer les vraies performances de notre modèle nous allons utiliser le critère de [validation croisee](https://fr.wikipedia.org/wiki/Validation_crois%C3%A9e) méthode **k-fold** (cross-validation).

In [42]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

from sklearn import linear_model
from sklearn import svm

#from sklearn.metrics import log_loss 
from sklearn.metrics import mean_squared_error
from math import sqrt


### Imputation des valeurs manquantes par la valeur "missing"

In [43]:
'''
X_train.categorie.fillna('missing', inplace=True)
X_test.categorie.fillna('missing', inplace=True)

X_train.etat.fillna('missing', inplace=True)
X_test.etat.fillna('missing', inplace=True)

X_train.nom_magasin.fillna('missing', inplace=True)
X_test.nom_magasin.fillna('missing', inplace=True)

X_train.vintage.fillna('missing', inplace=True)
X_test.vintage.fillna('missing', inplace=True)

X_train.sous_categorie_1.fillna('missing', inplace=True)
X_test.sous_categorie_1.fillna('missing', inplace=True)

X_train.taille.fillna('missing', inplace=True)
X_test.taille.fillna('missing', inplace=True)

X_train.garantie.fillna('missing', inplace=True)
X_test.garantie.fillna('missing', inplace=True)

X_train.couleur.fillna('missing', inplace=True)
X_test.couleur.fillna('missing', inplace=True)
'''

"\nX_train.categorie.fillna('missing', inplace=True)\nX_test.categorie.fillna('missing', inplace=True)\n\nX_train.etat.fillna('missing', inplace=True)\nX_test.etat.fillna('missing', inplace=True)\n\nX_train.nom_magasin.fillna('missing', inplace=True)\nX_test.nom_magasin.fillna('missing', inplace=True)\n\nX_train.vintage.fillna('missing', inplace=True)\nX_test.vintage.fillna('missing', inplace=True)\n\nX_train.sous_categorie_1.fillna('missing', inplace=True)\nX_test.sous_categorie_1.fillna('missing', inplace=True)\n\nX_train.taille.fillna('missing', inplace=True)\nX_test.taille.fillna('missing', inplace=True)\n\nX_train.garantie.fillna('missing', inplace=True)\nX_test.garantie.fillna('missing', inplace=True)\n\nX_train.couleur.fillna('missing', inplace=True)\nX_test.couleur.fillna('missing', inplace=True)\n"

### Encodage des features catégorielles

Les algorithmes de machine learning s'attendent à avoir en entrée des **nombres**, et non pas des chaînes de caractères. C'est pourquoi nous transformons les **features catégorielles** en nombres, à l'aide de ```LabelEncoder()```

In [44]:
X_train.region.unique()

array(['Свердловская область', 'Самарская область', 'Ростовская область',
       'Татарстан', 'Волгоградская область', 'Нижегородская область',
       'Пермский край', 'Оренбургская область', 'Ханты-Мансийский АО',
       'Тюменская область', 'Башкортостан', 'Краснодарский край',
       'Новосибирская область', 'Омская область', 'Белгородская область',
       'Челябинская область', 'Воронежская область', 'Кемеровская область',
       'Саратовская область', 'Владимирская область',
       'Калининградская область', 'Красноярский край',
       'Ярославская область', 'Удмуртия', 'Алтайский край',
       'Иркутская область', 'Ставропольский край', 'Тульская область'], dtype=object)

In [45]:
#X_train.deal_probability.unique()

In [46]:
X_train.parent_category_name.unique()

array(['Личные вещи', 'Для дома и дачи', 'Бытовая электроника',
       'Транспорт', 'Недвижимость', 'Животные', 'Хобби и отдых', 'Услуги',
       'Для бизнеса'], dtype=object)

In [47]:
X_train.category_name.unique()

array(['Товары для детей и игрушки', 'Мебель и интерьер', 'Аудио и видео',
       'Автомобили', 'Ремонт и строительство', 'Одежда, обувь, аксессуары',
       'Детская одежда и обувь', 'Квартиры', 'Товары для компьютера',
       'Собаки', 'Дома, дачи, коттеджи', 'Товары для животных',
       'Другие животные', 'Комнаты', 'Коллекционирование',
       'Коммерческая недвижимость', 'Посуда и товары для кухни',
       'Красота и здоровье', 'Аквариум', 'Телефоны', 'Часы и украшения',
       'Предложение услуг', 'Птицы', 'Спорт и отдых',
       'Музыкальные инструменты', 'Бытовая техника',
       'Игры, приставки и программы', 'Земельные участки',
       'Продукты питания', 'Кошки', 'Билеты и путешествия',
       'Книги и журналы', 'Растения', 'Гаражи и машиноместа',
       'Мотоциклы и мототехника', 'Планшеты и электронные книги',
       'Оборудование для бизнеса', 'Настольные компьютеры', 'Ноутбуки',
       'Велосипеды', 'Грузовики и спецтехника', 'Готовый бизнес',
       'Фототехника', 'Вод

In [48]:
X_train.activation_date.unique()

array(['2017-03-28', '2017-03-26', '2017-03-20', '2017-03-25',
       '2017-03-16', '2017-03-23', '2017-03-17', '2017-03-22',
       '2017-03-19', '2017-03-21', '2017-03-27', '2017-03-18',
       '2017-03-24', '2017-03-15', '2017-03-29', '2017-04-02',
       '2017-04-07', '2017-04-01', '2017-03-31', '2017-03-30', '2017-04-03'], dtype=object)

In [49]:
X_test.activation_date.unique()

array(['2017-04-18', '2017-04-16', '2017-04-17', '2017-04-15',
       '2017-04-12', '2017-04-13', '2017-04-14', '2017-04-19', '2017-04-20'], dtype=object)

In [50]:
X_train.user_type.unique()

array(['Private', 'Company', 'Shop'], dtype=object)

In [51]:
X_train["len_title"] = X_train.title.apply(lambda x : len(x))
X_test["len_title"] = X_test.title.apply(lambda x : len(x))

In [52]:
X_train["len_des"] = X_train.description.apply(lambda x : len(str(x)))
X_test["len_des"] = X_test.description.apply(lambda x : len(str(x)))

In [53]:
X_train["date"] = X_train.activation_date.apply(lambda x : int(x[-2:]))
X_test["date"] = X_test.activation_date.apply(lambda x : int(x[-2:]))

In [54]:
te_m = X_test["date"].min()
te_N = float(X_test["date"].max() - te_m)

tr_m = X_train["date"].min()
tr_N = float(X_train["date"].max() - tr_m)

X_train["date"] = X_train.date.apply(lambda x : (x-tr_m) / tr_N )
X_test["date"] = X_test.date.apply(lambda x : (x-te_m) / te_N )

In [55]:
le_user_type = LabelEncoder()
X_train['user_type'] = le_user_type.fit_transform(X_train.user_type)
X_test['user_type'] = le_user_type.transform(X_test.user_type)

In [56]:
le_category_name = LabelEncoder()
X_train['category_name'] = le_category_name.fit_transform(X_train.category_name)
X_test['category_name'] = le_category_name.transform(X_test.category_name)

In [57]:
le_parent_category_name = LabelEncoder()
X_train['parent_category_name'] = le_parent_category_name.fit_transform(X_train.parent_category_name)
X_test['parent_category_name'] = le_parent_category_name.transform(X_test.parent_category_name)

In [58]:
le_region = LabelEncoder()
X_train['region'] = le_region.fit_transform(X_train.region)
X_test['region'] = le_region.transform(X_test.region)

In [59]:
features = ["len_title", "user_type", "parent_category_name", "category_name", "date",  "len_des", "region"]
            

# , "parent_category_name", "category_name"
# , "date",  "len_des", "region"

In [60]:
X_train[features].head(5)

,len_title,user_type,parent_category_name,category_name,date,len_des,region
item_id,,,,,,,
b912c3c6a6ad,21,1,4,42,0.900000,58,19
2dac0150717d,17,1,2,22,0.833333,41,17
ba83aefab5dc,14,1,0,2,0.633333,99,16
02996f1dd2ea,10,0,4,42,0.800000,22,21
7c90be56d2ab,14,1,6,0,0.500000,24,4


In [61]:
X_test[features].head(5)

,len_title,user_type,parent_category_name,category_name,date,len_des,region
item_id,,,,,,,
6544e41a8817,15,1,4,10,0.750,12,4
65b9484d670f,16,1,8,5,0.500,88,19
8bab230b2ecd,3,1,0,2,0.625,132,12
8e348601fefc,17,1,2,4,0.625,93,18
8bd2fe400b89,17,1,4,42,0.375,60,14


In [62]:
y_train = X_train["deal_probability"]
#y_train = y_train.apply(lambda x : int(x*100))

In [63]:
#len(y_train.unique())

In [64]:
classifiers = [XGBClassifier(n_estimators=400, max_depth=6, n_jobs=-1)]
#    svm.SVR()
#    linear_model.LinearRegression()
#    linear_model.SGDRegressor(),
#    linear_model.BayesianRidge(),
#    linear_model.LassoLars(),

#    linear_model.TheilSenRegressor()
#    linear_model.ARDRegression(),


#    linear_model.PassiveAggressiveRegressor()

In [65]:
import lightgbm as lgb

dftrainLGB = lgb.Dataset(data = X_train[features], label = np.ravel(y_train), feature_name = features)

params = {'objective': 'regression'}

cv_results = lgb.cv(
        params,
        dftrainLGB,
        num_boost_round=100,
        nfold=5,
        metrics='mse',
        early_stopping_rounds=10,
        stratified=False
        )

In [66]:
cv_results

{'l2-mean': [0.065882298587904967,
  0.064454362992917083,
  0.063295421761926346,
  0.062346983188784254,
  0.06157187431169029,
  0.060933629998748032,
  0.060405688970826175,
  0.059971615206623349,
  0.059612881328530251,
  0.059316021969051225,
  0.059071397739671946,
  0.058866366481301016,
  0.058694784084321119,
  0.058548234198224733,
  0.058424920109943768,
  0.058320578714238833,
  0.05823069958037895,
  0.058153226717109541,
  0.058084137153788094,
  0.058026049172390293,
  0.057972197688181473,
  0.057924190599436487,
  0.057877920537486159,
  0.057837136794128628,
  0.057802428992846065,
  0.057772381342123237,
  0.057742757202604987,
  0.057716469409064719,
  0.057688817428737917,
  0.05766591338016562,
  0.057644571744448335,
  0.057622831994336696,
  0.057608652508838142,
  0.057591911966120769,
  0.057575187027755784,
  0.057561809443444044,
  0.05754393967647238,
  0.057531215100356969,
  0.05751845821503717,
  0.057506205305536809,
  0.057495335045395869,
  0.057485

In [ ]:
y_pred=clf.predict(x_test)

In [96]:
'''
model = linear_model.LinearRegression()
#model = XGBClassifier(n_estimators=400, max_depth=6, n_jobs=-1)
#model = RandomForestClassifier(n_estimators=300)

ppl = Pipeline([("imputer", Imputer(strategy='median')),
                ("clf", model)])

ppl.fit(X_train[features], np.ravel(y_train))   # .loc[:, features]

pred_train = ppl.predict(X_train[features])
pred_cv = cross_val_predict(ppl, X_train[features], np.ravel(y_train), method='predict', cv=5, n_jobs=-1)
'''

'\nmodel = linear_model.LinearRegression()\n#model = XGBClassifier(n_estimators=400, max_depth=6, n_jobs=-1)\n#model = RandomForestClassifier(n_estimators=300)\n\nppl = Pipeline([("imputer", Imputer(strategy=\'median\')),\n                ("clf", model)])\n\nppl.fit(X_train[features], np.ravel(y_train))   # .loc[:, features]\n\npred_train = ppl.predict(X_train[features])\npred_cv = cross_val_predict(ppl, X_train[features], np.ravel(y_train), method=\'predict\', cv=5, n_jobs=-1)\n'

In [ ]:

count_ = 0
for model in classifiers:
    ppl = Pipeline([("imputer", Imputer(strategy='median')),("clf", model)])
    ppl.fit(X_train[features], np.ravel(y_train))

    pred_train = ppl.predict(X_train[features])
    pred_cv = cross_val_predict(ppl, X_train[features], np.ravel(y_train),
                                method='predict', cv=5, n_jobs=-1)
    print('nb_model : ', count_)
    print("LogLoss sur echantillon train:",sqrt(mean_squared_error(y_pred=pred_train, y_true=y_train)))
    print("LogLoss sur echantillon train (CV):",sqrt(mean_squared_error(y_pred=pred_cv, y_true=y_train)))
    
    count_ +=1 


# Calcul de l'erreur: logloss

In [31]:
from sklearn.metrics import log_loss 
from sklearn.metrics import mean_squared_error
from math import sqrt

#rms = sqrt(mean_squared_error(y_actual, y_predicted))

In [32]:
print("LogLoss sur echantillon train:",sqrt(mean_squared_error(y_pred=pred_train, y_true=y_train)))
print("LogLoss sur echantillon train (CV):",sqrt(mean_squared_error(y_pred=pred_cv, y_true=y_train)))

LogLoss sur echantillon train: 0.26004724441077304
LogLoss sur echantillon train (CV): 0.2600474968465803


# Calcul des predictions

In [24]:
pred_test = ppl.predict(X_test[features])

In [25]:
df_submission = pd.DataFrame(pred_test, index=X_test.index)

## Possibilité n°2 : Soumettez un fichier CSV

1. Aller sur la plateforme [QScore](https://qscore.meilleurdatascientistdefrance.com) puis dans "Submissions" > "Submit with a file"
2. Déposer le fichier CSV

In [27]:
df_submission.to_csv("my_prediction_1.csv", index_label="item_id", header=["deal_probability"])